In [16]:
import pandas as pd
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
cwd = os.getcwd()
from utils import rename_columns,filter_by_year,filter_columns,process_data


##### Estimate the current utilisation of SH by sector

In [ ]:
# configure the current data 
renamed_columns_list = {
    "vaerk_id": "plant_id",
    "aar": "year",
    "anlaegstype_navn": "plant_type",
    "vaerk_kommune": "municipality_code",
    "varmeprod_TJ": "heat_production_TJ",
    "braendselsfrit_TJ": 'fuel_free_TJ',
    "vrktypeid": 'vrk_Id'
}

relevant_columns =['plant_id', 'year','municipality_code', 'sector','plant_type', 'heat_production_TJ','fuel_free_TJ','vrk_Id']
year=2022
# Read the data
df_raw = pd.read_excel('../data/EPT_DEA/6_ept_produktions-_og_forbrugsdata_2021-2023.xlsx', sheet_name='EPT2021-2023')
df_raw= rename_columns(df_raw,renamed_columns_list)

df_raw = filter_by_year(df_raw,year)
df_raw = filter_columns(df_raw,relevant_columns)
df_raw


In [ ]:
df_raw['heat_production_TJ'].sum()

In [ ]:
df = df_raw
df = df[( df['fuel_free_TJ'] >0 ) & (df['vrk_Id'] == 'ER')]
df

In [10]:
ndf =df.groupby( 'sector')['heat_production_TJ'].agg('sum')
ndf.to_excel('../data/processed/currentSH_bysector.xlsx')

In [11]:
municipality_df =df.groupby( ['municipality_code','sector'])['heat_production_TJ'].agg('sum')
municipality_df.to_excel('../data/processed/currentSH_bysector_bymunicipality.xlsx')

##### Estimate the Varmeplant technical potentials of SH by sector

In [ ]:
#a = gpd.read_file('../data/Varmeplan Danmark 2021/industrial_excess_heat_new_classes.geojson')
points = gpd.read_file('../data/Varmeplan Danmark 2021/points_in_DH.geojson')
#

In [ ]:
points.drop(columns=['SectorClas','SectorCl_1','sectorcl_2','sectorcl_3','geometry'],inplace=True)
points.head(3)

In [5]:
potentials =points.groupby('FiveClasses').agg({'GJ_samlet':'sum','GJ_over_80':'sum','GJ_60_80C':'sum','GJ_under_6':'sum' })
potentials.rename(columns={'GJ_samlet':'Total_GJ','GJ_over_80':'HighTemp_GJ','GJ_60_80C':'MediumTemp_GJ','GJ_under_6':'LowTemp_GJ'},inplace=True)
potentials.to_excel('../data/processed/varmplanSH_bysector.xlsx')

In [6]:
potentials_bymunicipality =points.groupby(['kommuneNav','kommuneKod','FiveClasses']).agg({'GJ_samlet':'sum','GJ_over_80':'sum','GJ_60_80C':'sum',	'GJ_under_6':'sum' })
potentials_bymunicipality.rename(columns={'GJ_samlet':'Total_GJ','GJ_over_80':'HighTemp_GJ','GJ_60_80C':'MediumTemp_GJ','GJ_under_6':'LowTemp_GJ'},inplace=True)

potentials_bymunicipality=potentials_bymunicipality.reset_index()
potentials_bymunicipality.to_excel('../data/processed/varmplanSH_bysector_bymunicipality.xlsx')

In [12]:
final=pd.concat([ndf.reset_index(),potentials.reset_index()], axis=1)

In [ ]:
final

In [26]:
#final.drop(columns=['sector'],inplace=True)
final['Total_DHproduction_GJ'] = df_raw['heat_production_TJ'].sum()*1000 
final['High temperature'] = final['HighTemp_GJ'] /final['Total_DHproduction_GJ'] *100
final['Medium temperature'] = final['MediumTemp_GJ'] /final['Total_DHproduction_GJ'] *100
final['Low temperature'] = final['LowTemp_GJ'] /final['Total_DHproduction_GJ'] *100
final['Current_GJ'] = final['heat_production_TJ'] *1000
final['Current utilisation'] = final['Current_GJ'] /final['Total_DHproduction_GJ']*100



In [ ]:
final= final[["FiveClasses",'Current utilisation', 'High temperature', 'Medium temperature','Low temperature']]

In [ ]:
# Set figure size and style
# Define custom colors for each category
colors = {
    "High temperature": "#8c6b72",  # Blue
    "Medium temperature": "#918ab5",  # Orange
    "Low temperature": "#c7c1e8",  # Green
    "Current utilisation": "#878f9c"  # Red
}

# Set figure size and style
plt.figure(figsize=(10, 8))
sns.set_style("whitegrid")

# Plot stacked bar chart with assigned colors
final.set_index("FiveClasses").plot(
    kind="bar", stacked=True, color=[colors[col] for col in final.columns[1:]], figsize=(10, 6)
)

# Labels and title
plt.xlabel("Sectors", fontsize=12)
plt.ylabel("Share of contribution (%)", fontsize=12)
#plt.title("Distribution of Heat Utilization Across Sectors", fontsize=14)
plt.xticks(rotation=45, ha="right")
plt.legend()
plt.tight_layout()

# Show plot
plt.show()